In [1]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum, min_, Var, LinExpr, max_

In [2]:
employment = pd.read_csv('D:/Documents/4004 Optimization/Project1/project1_new_datasets/new_employment.csv')
income = pd.read_csv('D:/Documents/4004 Optimization/Project1/project1_new_datasets/new_income.csv')
pop =  pd.read_csv('D:/Documents/4004 Optimization/Project1/project1_new_datasets/new_population.csv')
fac = pd.read_csv('D:/Documents/4004 Optimization/Project1/project1_new_datasets/new_child_care.csv')
loc = pd.read_csv('D:/Documents/4004 Optimization/Project1/project1_new_datasets/new_potential_loc.csv')

In [3]:
pop['10-12'] = (pop['10-14'] * 3 / 5).round().astype(int)

In [4]:
zip = income.merge(employment, how="left", left_on="zip_code", right_on="zip_code")
zip['is_high_demand'] = (zip['employment rate'] >= 0.6) | (zip['average income'] <= 60000)


In [5]:
slots = fac.groupby("zip_code").sum()[['infant_capacity', 'toddler_capacity', 'preschool_capacity', 'school_age_capacity', 'children_capacity', 'total_capacity']]
zip = zip.merge(slots, how="left", left_on="zip_code", right_on="zip_code")
zip = zip.merge(pop[['zip_code', '-5', '5-9', '10-12']], left_on="zip_code", right_on="zip_code")


In [6]:
zip['is_desert'] = (zip['is_high_demand'] & (zip['total_capacity'] <= 1/2 * (zip['-5'] + zip['5-9'] + zip['10-12']))) | \
                   (~zip['is_high_demand'] & (zip['total_capacity'] <= 1/3 * (zip['-5'] + zip['5-9'] + zip['10-12'])))
zip.head(20)

,zip_code,average income,employment rate,is_high_demand,infant_capacity,toddler_capacity,preschool_capacity,school_age_capacity,children_capacity,total_capacity,-5,5-9,10-12,is_desert
0,10001,102878.033603,0.595097,False,0,0,0,585,24,609,744,784,565,True
1,10002,59604.041165,0.520662,True,0,0,18,4508,203,4729,2142,3046,1919,False
2,10003,114273.049645,0.497244,False,0,0,0,1995,0,1995,1440,1034,572,False
3,10004,132004.310345,0.506661,False,0,0,0,263,0,263,433,182,97,False
4,10005,121437.713311,0.665833,True,0,0,0,39,0,39,484,204,137,True
5,10006,126377.118644,0.631692,True,0,0,14,142,0,156,128,96,45,False
6,10007,138853.904282,0.528910,False,0,0,0,284,0,284,605,451,174,True
7,10009,77133.233533,0.514567,False,0,0,18,1660,106,1784,1896,1658,1101,False
8,10010,116272.698810,0.492749,False,0,0,0,234,0,234,1422,1592,569,True
9,10011,120420.792079,0.557000,False,0,0,42,1908,6,1956,1209,1200,818,False


In [7]:
#print(pop)
fac = fac.merge(pop,how="left",left_on="zip_code", right_on="zip_code")
#test = fac.groupby('facility_id').count()
#print(test[test["zip_code"]>1])
fac

,zip_code,facility_id,program_type,facility_status,facility_name,city,school_district_name,infant_capacity,toddler_capacity,preschool_capacity,...,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,10-12
0,10001,837597,SACC,Registration,I Have a Dream Foundation,New York,Manhattan 2,0,0,0,...,1903,1704,1225,1323,933,815,616,488,576,565
1,10001,661697,GFDC,License,Chelsea Little Angels Day Care,New York,Manhattan 2,0,0,0,...,1903,1704,1225,1323,933,815,616,488,576,565
2,10001,837329,SACC,Registration,Bright Horizons at Hudson Yards,New York,Manhattan 2,0,0,0,...,1903,1704,1225,1323,933,815,616,488,576,565
3,10001,350076,FDC,Registration,GRAMMAS HANDS,New York,Manhattan 2,0,0,0,...,1903,1704,1225,1323,933,815,616,488,576,565
4,10001,292419,SACC,Registration,The Hudson Guild @26th Street,New York,Manhattan 2,0,0,0,...,1903,1704,1225,1323,933,815,616,488,576,565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14751,14770,115279,SACC,Registration,Young Men's Christian Association of Olean NY ...,Portville,Portville,0,0,15,...,218,210,177,181,120,191,83,80,41,134
14752,14772,782407,SACC,Registration,"The Relief Zone, Inc.",Randolph,Randolph,0,0,32,...,334,238,272,311,306,170,196,55,58,134
14753,14772,888218,GFDC,License,Main Street Munchkins,Randolph,Randolph,0,0,0,...,334,238,272,311,306,170,196,55,58,134
14754,14805,866946,FDC,Registration,Alpine Children's House,Alpine,Trumansburg,0,0,0,...,31,29,143,105,36,141,12,8,10,17


In [8]:
fac = fac[fac['total_capacity'] > 0]


In [9]:
zip['demand_child'] = (zip['-5'] + zip['5-9'] + zip['10-12']).round().astype(int)
zip['under5_capacity'] = zip['infant_capacity'] + zip['toddler_capacity'] + zip['preschool_capacity']
fac['under5_capacity'] = fac['infant_capacity'] + fac['toddler_capacity'] + fac['preschool_capacity']


<ipython-input-9-c8bd8d379e9a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fac['under5_capacity'] = fac['infant_capacity'] + fac['toddler_capacity'] + fac['preschool_capacity']


In [10]:
zip = zip[zip['demand_child'] > 0]


In [11]:
new_fac_cost = {
    'size': ['small', 'medium', 'large'],
    'total_slots': [100, 200, 400],
    'under5_slots': [50, 100, 200],
    'cost': [65000, 95000, 115000]
}
new_fac_cost = pd.DataFrame(new_fac_cost)
new_fac_cost

,size,total_slots,under5_slots,cost
0,small,100,50,65000
1,medium,200,100,95000
2,large,400,200,115000


In [12]:
zipcode = zip['zip_code'].tolist()
new_fac_size = ['small', 'medium', 'large']
exist_fac = fac[fac['total_capacity'] <= 500]['facility_id'].unique()
age = ['under5', 'over5']

# Q1

In [38]:
m = Model("child_solution")

In [39]:
# Decision Variables：new facilities & slot expansion
x = {}
for i in zipcode:
    for j in new_fac_size:
        x[i, j] = m.addVar(vtype = GRB.INTEGER, name = f"x_{i}_{j}")

y = {}
for f in exist_fac:
    for a in age:
        y[f, a] = m.addVar(vtype = GRB.INTEGER, name = f"y_{f}_{a}")


In [40]:
# Objective Function：Min the cost of...
new_fac_cost_expr = quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['cost'].values[0] for i in zipcode for j in new_fac_size)
expand_cost_expr = sum(((20000 + (fac[fac['facility_id'] == f]['total_capacity'].values[0]) * 200) * (y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] + 100 * y[f, 'under5']) for f in exist_fac)

m.setObjective(new_fac_cost_expr + expand_cost_expr, GRB.MINIMIZE)


In [43]:
# Constraints:
# (1) no desert in both high demand and normal demand region
for i in zipcode:
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == i]['total_capacity'].values[0]) / zip[zip['zip_code'] == i]['demand_child'].values[0] >=
        1/2 * zip[zip['zip_code'] == i]['is_high_demand'].values[0], f"no_desert_high_demand_{i}")
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == i]['total_capacity'].values[0]) / zip[zip['zip_code'] == i]['demand_child'].values[0] >=
        1/3, f"no_desert_low_demand_{i}")


In [44]:
# (2) under5 take up a higher coverage percantage
for i in zipcode:
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for j in new_fac_size) +
         quicksum(y[f, 'under5'] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == i]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == i]['-5'].values[0], f"under5_demand_{i}")


In [45]:
# (3) expansion maximum scale
for f in exist_fac:
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) <= 0.20 * fac[fac['facility_id'] == f]['total_capacity'].values[0], f"expansion_restriction1_{f}")
    m.addConstr(y[f, 'under5'] + y[f, 'over5'] + fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 500 , f"expansion_restriction2_{f}")

In [46]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19042.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 32560 rows, 32542 columns and 142312 nonzeros
Model fingerprint: 0x65f1cd99
Variable types: 0 continuous, 32542 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-05, 2e+02]
  Objective range  [2e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [9e-05, 8e+03]
Found heuristic solution: objective 3.855950e+08
Presolve removed 32521 rows and 32465 columns
Presolve time: 1.81s
Presolved: 39 rows, 77 columns, 191 nonzeros
Found heuristic solution: objective 3.400428e+08
Variable types: 0 continuous, 77 integer (28 binary)

Root relaxation: interrupted, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth In

In [47]:
if m.status == GRB.OPTIMAL:
    print("Optimal solution found. Results:")
    
    new_facilities_data = []
    
    for i in zipcode:
        for j in new_fac_size:
            if x[i, j].x > 0:
                new_facilities_data.append({'zip_code': i, 'size': j, 'Num_new_facilities': x[i, j].x})
                print(f"Region {i}: Build {x[i, j].x} {j} size new facilities.")
                
    
    new_facilities_df = pd.DataFrame(new_facilities_data)

    expand_slots_data = []
    
    for f in exist_fac:
        for a in age:
            if y[f, a].x > 0:
                expand_slots_data.append({'facility_id': f, 'age_group': a, 'Num_expand_slots': y[f, a].x})
                print(f"Facility {f}: Expand capacity for age group {a} by {y[f, a].x}.")
    
    expand_slots_df = pd.DataFrame(expand_slots_data)

elif m.status == GRB.INFEASIBLE:
    print("The model is infeasible. Check constraints or data.")
elif m.status == GRB.UNBOUNDED:
    print("The model is unbounded. Consider revising the model.")
else:
    print(f"Optimization was stopped with status {m.status}")


Optimal solution found. Results:
Region 10001: Build 2.0 large size new facilities.
Region 10002: Build 3.0 large size new facilities.
Region 10003: Build 4.0 large size new facilities.
Region 10004: Build 1.0 small size new facilities.
Region 10004: Build 1.0 large size new facilities.
Region 10005: Build 2.0 large size new facilities.
Region 10006: Build 1.0 small size new facilities.
Region 10007: Build 2.0 large size new facilities.
Region 10009: Build 5.0 large size new facilities.
Region 10010: Build 5.0 large size new facilities.
Region 10011: Build 3.0 large size new facilities.
Region 10012: Build 1.0 small size new facilities.
Region 10012: Build 2.0 large size new facilities.
Region 10013: Build 4.0 large size new facilities.
Region 10014: Build 2.0 large size new facilities.
Region 10016: Build 6.0 large size new facilities.
Region 10017: Build 1.0 large size new facilities.
Region 10019: Build 1.0 small size new facilities.
Region 10019: Build 3.0 large size new facilities

Facility 769197: Expand capacity for age group under5 by 35.0.
Facility 160975: Expand capacity for age group under5 by 16.0.
Facility 771628: Expand capacity for age group under5 by 13.0.
Facility 42220: Expand capacity for age group under5 by 22.0.
Facility 910210: Expand capacity for age group under5 by 26.0.
Facility 896205: Expand capacity for age group under5 by 19.0.
Facility 910213: Expand capacity for age group under5 by 27.0.
Facility 897658: Expand capacity for age group under5 by 28.0.
Facility 908651: Expand capacity for age group under5 by 37.0.
Facility 910211: Expand capacity for age group under5 by 26.0.
Facility 40119: Expand capacity for age group under5 by 16.0.
Facility 239587: Expand capacity for age group under5 by 24.0.
Facility 191359: Expand capacity for age group under5 by 2.0.
Facility 556796: Expand capacity for age group under5 by 13.0.
Facility 396346: Expand capacity for age group under5 by 3.0.
Facility 842902: Expand capacity for age group under5 by 4.

Facility 449424: Expand capacity for age group under5 by 17.0.
Facility 301509: Expand capacity for age group under5 by 27.0.
Facility 555934: Expand capacity for age group under5 by 22.0.
Facility 255987: Expand capacity for age group under5 by 1.0.
Facility 870417: Expand capacity for age group under5 by 27.0.
Facility 851059: Expand capacity for age group under5 by 7.0.
Facility 435681: Expand capacity for age group under5 by 3.0.
Facility 251902: Expand capacity for age group under5 by 3.0.
Facility 793108: Expand capacity for age group under5 by 15.0.
Facility 791040: Expand capacity for age group under5 by 1.0.
Facility 890544: Expand capacity for age group under5 by 3.0.
Facility 896968: Expand capacity for age group under5 by 1.0.
Facility 901366: Expand capacity for age group under5 by 1.0.
Facility 898780: Expand capacity for age group under5 by 3.0.
Facility 885703: Expand capacity for age group under5 by 1.0.
Facility 891629: Expand capacity for age group under5 by 3.0.
Fac

Facility 837691: Expand capacity for age group under5 by 19.0.
Facility 733706: Expand capacity for age group under5 by 11.0.
Facility 43114: Expand capacity for age group under5 by 2.0.
Facility 375358: Expand capacity for age group under5 by 25.0.
Facility 105304: Expand capacity for age group under5 by 22.0.
Facility 794603: Expand capacity for age group under5 by 3.0.
Facility 891454: Expand capacity for age group under5 by 14.0.
Facility 42849: Expand capacity for age group under5 by 5.0.
Facility 39761: Expand capacity for age group under5 by 34.0.
Facility 803420: Expand capacity for age group under5 by 15.0.
Facility 299068: Expand capacity for age group under5 by 18.0.
Facility 39760: Expand capacity for age group under5 by 48.0.
Facility 348495: Expand capacity for age group under5 by 32.0.
Facility 40941: Expand capacity for age group under5 by 14.0.
Facility 600978: Expand capacity for age group under5 by 16.0.
Facility 39762: Expand capacity for age group under5 by 18.0.
F

In [48]:
new_facilities_df

,zip_code,size,Num_new_facilities
0,10001,large,2.0
1,10002,large,3.0
2,10003,large,4.0
3,10004,small,1.0
4,10004,large,1.0
...,...,...,...
997,14767,medium,1.0
998,14770,medium,1.0
999,14772,large,1.0
1000,14805,small,1.0


In [49]:
expand_slots_df

,facility_id,age_group,Num_expand_slots
0,837597,under5,16.0
1,292419,under5,5.0
2,229433,under5,17.0
3,893683,under5,58.0
4,120981,under5,61.0
...,...,...,...
2085,39768,under5,14.0
2086,705760,under5,9.0
2087,868973,under5,2.0
2088,42583,under5,23.0


In [28]:
new_facilities_df.to_excel('new_facilities_df.xlsx')
expand_slots_df.to_excel('expand_slots_df.xlsx')

<ipython-input-28-f82315cead8b>:1: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  new_facilities_df.to_excel('new_facilities_df.xlsx')
<ipython-input-28-f82315cead8b>:2: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.8' currently installed).
  expand_slots_df.to_excel('expand_slots_df.xlsx')


In [24]:
#需要新建的地区数量
print(len(new_facilities_df['zip_code'].unique()))
#总新建数量

#每个size新建的数量
print(new_facilities_df.groupby('size')['Num_new_facilities'].sum())
#top10 新建数量的地区



886
size
large     2425.0
medium     183.0
small      245.0
Name: Num_new_facilities, dtype: float64


## Q3

In [13]:
from haversine import haversine, Unit

def loc_distance(data1, data2, m, n):
    lat1 = data1.loc[m, 'latitude']
    lon1 = data1.loc[m, 'longitude']
    lat2 = data2.loc[n, 'latitude']
    lon2 = data2.loc[n, 'longitude']
    distance = haversine((lat1, lon1), (lat2, lon2), unit=Unit.MILES)
    return distance

C:\Users\zhurt\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [14]:
m = Model('child_solution_q3')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-10


In [15]:
# Variable
x = {}
for i in range(len(loc)):
    for j in new_fac_size:
        x[i, j] = m.addVar(vtype = GRB.BINARY, name = f'x_{i}_{j}')
        
y = {}
for f in exist_fac:
    for a in age:
        y[f, a] = m.addVar(vtype = GRB.INTEGER, name = f'y_{f}_{a}')

steps = ['step2', 'step3']
z = {}
for f in exist_fac:
    for s in steps:
        z[f, s] = m.addVar(vtype = GRB.BINARY, name = f'step_{f}_{s}')
        
tf1 = {}
tf2 = {}
tf3 = {}
tf2_super = {}
tf3_super = {}
for f in exist_fac:
    tf1[f] = m.addVar(vtype = GRB.CONTINUOUS, lb = -1000, name = f'total1_{f}')
    tf2[f] = m.addVar(vtype = GRB.CONTINUOUS, lb = -1000, name = f'total2_{f}')
    tf3[f] = m.addVar(vtype = GRB.CONTINUOUS, lb = -1000, name = f'total3_{f}')
    tf2_super[f] = m.addVar(vtype = GRB.CONTINUOUS, lb = -1000, name = f'total2_super_{f}')
    tf3_super[f] = m.addVar(vtype = GRB.CONTINUOUS, lb = -1000, name = f'total3_super_{f}')

In [16]:
# Objectives
all_coverage = (quicksum(x[i, 'small'] * 100 + x[i, 'medium'] * 200 + x[i, 'large'] * 400 for i in range(len(loc))) +
                quicksum(y[f, a] for f in exist_fac for a in age) +
                zip['total_capacity'].sum()) / zip['demand_child'].sum()
under5_coverage = (quicksum(x[i, 'small'] * 50 + x[i, 'medium'] * 100 + x[i, 'large'] * 200 for i in range(len(loc))) +
                quicksum(y[f, 'under5'] for f in exist_fac) +
                zip['under5_capacity'].sum()) / zip['-5'].sum()
weighted_coverage_rate = (2/3) * under5_coverage + (1/3) * all_coverage

m.setObjective(weighted_coverage_rate, GRB.MAXIMIZE)

In [17]:
# Constraints
#(1) no desert
for code in zipcode:
    i_ = loc[loc['zip_code'] == code].index.tolist()
    
    # 计算公共部分
    common_expression = (
        quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
        quicksum(y[f, a] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist() for a in age) +
        zip[zip['zip_code'] == code]['total_capacity'].values[0]
    ) / zip[zip['zip_code'] == code]['demand_child'].values[0]
    
    # 添加约束
    m.addConstr(common_expression >= 1/2 * zip[zip['zip_code'] == code]['is_high_demand'].values[0], 
                 f"no_desert_high_demand_{code}")
    
    m.addConstr(common_expression >= 1/3, 
                 f"no_desert_low_demand_{code}")

In [18]:
#(2) under5
for code in zipcode:
    i_ = loc[loc['zip_code'] == code].index.tolist()
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(y[f, 'under5'] for f in fac[(fac['zip_code'] == i) & (fac['total_capacity'] <= 500) ]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == code]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == code]['-5'].values[0], f"under5_demand_{code}")

# 未化简
# Constraints
#(1) no desert
for code in zipcode:
    i_ = loc[loc['zip_code'] == code].index.tolist()
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == code]['total_capacity'].values[0]) / zip[zip['zip_code'] == code]['demand_child'].values[0] >=
        1/2 * zip[zip['zip_code'] == code]['is_high_demand'].values[0], f"no_desert_high_demand_{code}")
    
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(y[f, a] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist() for a in age) +
         zip[zip['zip_code'] == code]['total_capacity'].values[0]) / zip[zip['zip_code'] == code]['demand_child'].values[0] >=
        1/3, f"no_desert_low_demand_{code}")
    
#(2) under5
for code in zipcode:
    i_ = loc[loc['zip_code'] == code].index.tolist()
    m.addConstr(
        (quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] == j]['under5_slots'].values[0] for i in i_ for j in new_fac_size) +
         quicksum(y[f, 'under5'] for f in fac[fac['zip_code'] == code]['facility_id'].unique().tolist()) +
         zip[zip['zip_code'] == code]['under5_capacity'].values[0]) >=
        2/3 * zip[zip['zip_code'] == code]['-5'].values[0], f"under5_demand_{code}")

In [19]:
#(3) expansion maximum + new facility limit
for f in exist_fac:
    m.addConstr(quicksum(y[f, a] for a in age) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.20, f'expansion_restriction_{f}')
    m.addConstr(quicksum(y[f, a] for a in age) + fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 500, f'expansion_restriction_{f}')

for i in range(len(loc)):
    m.addConstr(quicksum(x[i, j] for j in new_fac_size) <= 1, f'new_facility_num_{i}')

m.update()

In [20]:
#(4) total cost less than $1 billion
c1 = {}
c2 = {}
c3 = {}
for f in exist_fac:
    c1[f] = m.addVar(vtype = GRB.CONTINUOUS, name = f'c1_{f}')
    c2[f] = m.addVar(vtype = GRB.CONTINUOUS, name = f'c2_{f}')
    c3[f] = m.addVar(vtype = GRB.CONTINUOUS, name = f'c3_{f}')
    

    
    m.addConstr(tf1[f] == y[f, 'under5'] + y[f, 'over5'])
    m.addConstr(tf2[f] == y[f, 'under5'] + y[f, 'over5'] - 0.1 * fac[fac['facility_id'] == f]['total_capacity'].values[0])
    m.addConstr(tf3[f] == y[f, 'under5'] + y[f, 'over5'] - 0.15 * fac[fac['facility_id'] == f]['total_capacity'].values[0])
    
    m.update()
    
    m.addConstr(tf2_super[f] == max_(tf2[f], constant = 0))
    m.addConstr(tf3_super[f] == max_(tf3[f], constant = 0))
    
    m.addConstr(c1[f] == min_(tf1[f], constant = 0.1 * fac[fac['facility_id'] == f]['total_capacity'].values[0]))
    m.addConstr(c2[f] == min_(tf2[f], constant =  0.05 * fac[fac['facility_id'] == f]['total_capacity'].values[0]))
    m.addConstr(c3[f] == min_(tf3[f], constant =  0.05 * fac[fac['facility_id'] == f]['total_capacity'].values[0]))

    m.update()

In [21]:
new_fac_cost_expr = quicksum(x[i, j] * new_fac_cost[new_fac_cost['size'] ==j]['cost'].values[0] for i in range(len(loc)) for j in new_fac_size)

expand_cost_expr = quicksum(
                      (20000 * c1[f]/fac[fac['facility_id'] == f]['total_capacity'].values[0] + 200 * c1[f])  +
                      (20000 * c2[f]/fac[fac['facility_id'] == f]['total_capacity'].values[0] + 400 * c2[f]) * z[f, 'step2'] +
                      (20000 * c3[f]/fac[fac['facility_id'] == f]['total_capacity'].values[0] + 1000 * c3[f]) * z[f, 'step3'] +
                      y[f, 'under5'] * 100
                      for f in exist_fac) 

m.addConstr(new_fac_cost_expr + expand_cost_expr <= 1000000000, 'total_cost')
m.update()

In [22]:
#(5) constraint for var z
for f in exist_fac:
    total_capacity = fac[fac['facility_id'] == f]['total_capacity'].values[0]
    #total_y = y[f, 'under5'] + y[f, 'over5']
     
    # 公共表达式
    #ratio = total_y / total_capacity
    ratio_1 = c1[f] / total_capacity
    ratio_2 = c2[f] / total_capacity
    ratio_3 = c3[f] / total_capacity

    # 添加约束
    m.addConstr(ratio_1 <= 0.1 , 'step1_upper_limit')
    m.addConstr(ratio_1 >= 0.1 * z[f, 'step2'], 'step1_lower_limit')
    m.addConstr(ratio_2 <= 0.05 * z[f, 'step2'], 'step2_upper_limit')
    m.addConstr(ratio_2 >= 0.05 * z[f, 'step3'], 'step2_lower_limit')
    m.addConstr(ratio_3 <= 0.05 * z[f, 'step3'], 'step3_upper_limit')
    m.addConstr(ratio_3 >= 0 , 'step3_lower_limit')


#(5) constraint for var z
for f in exist_fac:
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.1 + (1 - z[f, 'step1']) * 1000, 'step1_limit')
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] >= 0.1 + z[f, 'step2'], 'step2_limit')
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.15 + (1 - z[f, 'step2']) * 1000, 'step2_upper_limit')
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] >= 0.15 + z[f, 'step3'], 'step3_limit')
    m.addConstr((y[f, 'under5'] + y[f, 'over5']) / fac[fac['facility_id'] == f]['total_capacity'].values[0] <= 0.2 + (1 - z[f, 'step3']) * 1000, 'step3_upper_limit')
    

In [23]:
#(6) distance of any two less than 0.06 mile
for code in zipcode:
    
    locations_in_zipcode = loc[loc['zip_code'] == code].index.tolist()
    
    exist_facilities_in_zipcode = fac[(fac['zip_code'] == code) & (fac['latitude'].notna()) & (fac['longitude'].notna())].index.tolist() 
    
    # 计算每个位置的设施数量
    x_sum = {l: quicksum(x[l, j] for j in new_fac_size) for l in locations_in_zipcode}

    for l1 in locations_in_zipcode:
        # 和新增的比较
        for l2 in locations_in_zipcode:
            if l1 < l2:
                m.addConstr(
                    loc_distance(loc, loc, l1, l2) <= 0.06 + (1 - x_sum[l1]) * 1000 + (1 - x_sum[l2]) * 1000,
                    name=f'distance_{l1}_{l2}'
                )
        
        #和原有的比较
        for l_exist_facility in exist_facilities_in_zipcode:

            m.addConstr(
                loc_distance(loc, fac, l1, l_exist_facility) <= 0.06 + (1 - x_sum[l1]) * 1000,
                name=f'distance_{l1}_{l_exist_facility}'
            )


#(6) distance of any two less than 0.6 mile
for code in zipcode:
    locations_in_zipcode = loc[loc['zip_code'] == code].index.tolist()
    for l1 in locations_in_zipcode:
        for l2 in locations_in_zipcode:
            if l1 < l2:
                m.addConstr(loc_distance(l1, l2) <= 0.6 + (1 - quicksum(x[l1, j] for j in new_fac_size)) * 1000 + (1 - quicksum(x[l2, j] for j in new_fac_size)) * 1000, name = 'distance_{l1}_{l2}')


In [28]:
#(7) fairness
zip_capacity = zip.set_index('zip_code')['total_capacity'].to_dict()
demand_child = zip.set_index('zip_code')['demand_child'].to_dict()

ratios = {}
for code in zipcode:
    i = loc[loc['zip_code'] == code].index.tolist()
    ratio = (
        quicksum(x[i_index, j] * new_fac_cost[new_fac_cost['size'] == j]['total_slots'].values[0] 
                 for i_index in i for j in new_fac_size) +
        quicksum(y[f, a] for f in fac[(fac['zip_code'] == code) & (fac['total_capacity'] <= 500) ]['facility_id'].unique() for a in age) +
        zip_capacity[code]
    ) / demand_child[code]
    ratios[code] = ratio

for code_a in zipcode:
    for code_b in zipcode:
        if code_a < code_b:
            ratio_a = ratios[code_a]
            ratio_b = ratios[code_b]
            m.addConstr(ratio_a - ratio_b <= 0.1, name=f'fair_ratio_positive_{code_a}_{code_b}')
            m.addConstr(ratio_a - ratio_b >= -0.1, name=f'fair_ratio_negative_{code_a}_{code_b}')


In [29]:
# Solve
m.setParam('NodeFileStart', 0.2)

m.optimize()

Set parameter NodefileStart to value 0.2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19042.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7778391 rows, 483792 columns and 719825531 nonzeros
Model fingerprint: 0xcaf98274
Model has 1 quadratic constraint
Model has 73705 general constraints
Variable types: 117928 continuous, 365864 integer (336382 binary)
Coefficient statistics:
  Matrix range     [4e-05, 1e+03]
  QMatrix range    [4e+02, 8e+03]
  QLMatrix range   [1e+02, 1e+05]
  Objective range  [1e-07, 2e-04]
  Bounds range     [1e+00, 1e+03]
  RHS range        [3e-17, 8e+03]
  QRHS range       [1e+09, 1e+09]
  GenCon const rng [2e-01, 5e+01]

Explored 0 nodes (0 simplex iterations) in 41.95 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Best objective -, best bound -, gap -

GurobiError: Out of memory

In [ ]:
#expansion cost (2)
# infeasible可能是有些zipcode能建的不多或者很多